In [ ]:
import os,json
import subprocess
import shutil
from concurrent.futures import ProcessPoolExecutor
from Utils.CModuleParcer import CModule
def c2preprocessed(c_str,count):
    c_path = f'{c_directory}/file{count}.c'
    open(c_path,'w').write(c_str)
    wasm_path = f'{wasm_directory}/file{count}.wasm'
    wat_path = f'{wat_directory}/file{count}.wat'
    cmodule = CModule(c_str)
    funclist = cmodule.get_ordered_funcs()
    if 'main' in funclist:
        import_str = ','.join([f'_{i}' for i in funclist])
        emcc_compile = f'emcc {c_path} -g -s EXPORTED_FUNCTIONS="{import_str}" -o {wasm_path}'
    else:
        emcc_compile = f'emcc {c_path} -g -s -c -o {wasm_path}'
    try:
        command = f'''{emcc_compile} ;\
            wasm2wat {wasm_path} -o {wat_path}'''
        subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    except:
        print(c_path)

def multi_c2wasm2wat(file_list, max_workers=100):
    print(f"一共{len(file_list)}个c文件")
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        for i, file_str in enumerate(file_list):
            executor.submit(c2preprocessed, file_str, i)
root_dir = '/'.join(os.getcwd().split('/')[:-1])
eval_dir = f'{root_dir}/Dataset/eval_dataset'
eval_data_path = f'{eval_dir}/eval_data.json'
with open(eval_data_path,'r') as f:
    eval_data_list = json.load(f)
c_directory = eval_dir + "/c"
if not os.path.exists(c_directory):
    os.makedirs(c_directory)
wasm_directory = eval_dir + "/wasm"
if not os.path.exists(wasm_directory):
    os.makedirs(wasm_directory)
wat_directory = eval_dir + "/wat"
if not os.path.exists(wat_directory):
    os.makedirs(wat_directory)

multi_c2wasm2wat(eval_data_list)